# Spark MLlib

- MLlib 是基于 spark-dataframe 构建的

In [4]:
import findspark
findspark.init("/Users/xinby/Library/Spark")


from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    master("local[*]").\
    config("spark.executor.memory", "4g").\
    config("spark.driver.memory", "4g").\
    config("spark.ui.showConsoleProgress", "false").\
    appName("MLlib").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[*] appName=MLlib>


### 数据读取

我们依然使用弹幕数据：

In [25]:
# cids = [144541892, 144541943, 160377038, 148952771, 150894103, 153392221, 156629080, 159982308, 162395026]
cids = [144541892, 144541943, 160377038, 148952771, 150894103]
jsons = [f"data/lec14-danmu-{cid}.json" for cid in cids]
jsons

['data/lec14-danmu-144541892.json',
 'data/lec14-danmu-144541943.json',
 'data/lec14-danmu-160377038.json',
 'data/lec14-danmu-148952771.json',
 'data/lec14-danmu-150894103.json']

In [30]:
df = spark.read.json(jsons, multiLine=True)
df.count()


11574000

出于演示目的，我们随机抽取一小部分数据并缓存：

In [31]:
df_small = df.sample(withReplacement=False, fraction=0.001, seed=123)
df_small.cache()
df_small.show(n=15)
df_small.count()

+---------+----------------------------------+----------+-----------------+----+----------+
|      cid|                           content|      date|               id|mode| post_time|
+---------+----------------------------------+----------+-----------------+----+----------+
|160377038|                            脱水！|2020-01-28|29497785794953219|   1|1580223858|
|160377038|                        技术公有化|2020-01-28|29497829588205573|   1|1580217034|
|160377038|                              脱水|2020-01-28|29497829597642757|   1|1580217024|
|160377038|                              浸泡|2020-01-28|29497833691807747|   1|1580216633|
|160377038|                          空位是？|2020-01-28|29497835948343299|   1|1580216368|
|160377038|                    你也会空中劈叉|2020-01-28|29497842766184453|   5|1580215732|
|160377038|                              脱水|2020-01-29|29497694497013763|   1|1580310440|
|160377038|                    让  我  先  跑|2020-01-29|29497730623078403|   5|1580281402|
|160377038|      

11500

### 数据转换与特征提取

除了 PySpark 自带的变换操作，我们还可以直接编写 Python 函数（基于 Pandas）对数据进行变换，如对弹幕进行分词。

In [34]:
import jieba
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def seg_words(danmu):
    # 弹幕通常包含大量空格以醒目，但破坏语义，先移除
    danmu = danmu.replace(" ", "")
    # 生成一个含有切分后词语的迭代器
    cut = jieba.lcut(danmu, cut_all=False)
    # 去掉多余的空格
    cut = filter(lambda x: x != " ", cut)
    # 再用空格将词语合并
    return " ".join(cut)

seg_words_udf = udf(seg_words, StringType())

In [35]:
df_seg = df_small.withColumn("seg", seg_words_udf(df_small.content))
df_seg.cache()
df_seg.show(n=10)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/wk/fm3ylspx3ml0t03x529wz0n80000gn/T/jieba.cache
Loading model cost 0.878 seconds.
Prefix dict has been built successfully.


+---------+--------------+----------+-----------------+----+----------+------------------+
|      cid|       content|      date|               id|mode| post_time|               seg|
+---------+--------------+----------+-----------------+----+----------+------------------+
|160377038|        脱水！|2020-01-28|29497785794953219|   1|1580223858|           脱水 ！|
|160377038|    技术公有化|2020-01-28|29497829588205573|   1|1580217034|       技术 公有化|
|160377038|          脱水|2020-01-28|29497829597642757|   1|1580217024|              脱水|
|160377038|          浸泡|2020-01-28|29497833691807747|   1|1580216633|              浸泡|
|160377038|      空位是？|2020-01-28|29497835948343299|   1|1580216368|        空位 是 ？|
|160377038|你也会空中劈叉|2020-01-28|29497842766184453|   5|1580215732|你 也 会 空中 劈叉|
|160377038|          脱水|2020-01-29|29497694497013763|   1|1580310440|              脱水|
|160377038|让  我  先  跑|2020-01-29|29497730623078403|   5|1580281402|       让 我 先 跑|
|160377038|    组织生活会|2020-01-29|29497759658672133|   1|15

接下来使用 MLlib 中的 `Tokenizer` 转换器来将词语转为列表：

In [36]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tok = Tokenizer(inputCol="seg", outputCol="words")
df_tok = tok.transform(df_seg)
df_tok.select("seg", "words").show(n=10)

+------------------+------------------------+
|               seg|                   words|
+------------------+------------------------+
|           脱水 ！|              [脱水, ！]|
|       技术 公有化|          [技术, 公有化]|
|              脱水|                  [脱水]|
|              浸泡|                  [浸泡]|
|        空位 是 ？|          [空位, 是, ？]|
|你 也 会 空中 劈叉|[你, 也, 会, 空中, 劈叉]|
|              脱水|                  [脱水]|
|       让 我 先 跑|        [让, 我, 先, 跑]|
|       组织生活 会|          [组织生活, 会]|
|         当场 揭穿|            [当场, 揭穿]|
+------------------+------------------------+
only showing top 10 rows



移除停用词：

In [37]:
stop_words = ["的", "了", "是", "，", "。", "？", "！", "：", "（", "）", "“", "”", ".", "…", "."]
rmstop = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=stop_words)
df_rmstop = rmstop.transform(df_tok)
df_rmstop.select("words", "filtered").show(n=10)

+------------------------+------------------------+
|                   words|                filtered|
+------------------------+------------------------+
|              [脱水, ！]|                  [脱水]|
|          [技术, 公有化]|          [技术, 公有化]|
|                  [脱水]|                  [脱水]|
|                  [浸泡]|                  [浸泡]|
|          [空位, 是, ？]|                  [空位]|
|[你, 也, 会, 空中, 劈叉]|[你, 也, 会, 空中, 劈叉]|
|                  [脱水]|                  [脱水]|
|        [让, 我, 先, 跑]|        [让, 我, 先, 跑]|
|          [组织生活, 会]|          [组织生活, 会]|
|            [当场, 揭穿]|            [当场, 揭穿]|
+------------------------+------------------------+
only showing top 10 rows



进而使用 `CountVectorizer` 来统计词频：

In [38]:
from pyspark.ml.feature import CountVectorizer, IDF

# minDF 表示进入字典的词最少需要在多少句子（弹幕）中出现
# vocabSize 表示取词频前几位的词语作为字典
counter = CountVectorizer(inputCol="filtered", outputCol="features",
                          minDF=10, vocabSize=500)

counter_model = counter.fit(df_rmstop)
print(counter_model.vocabulary)

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wk/fm3ylspx3ml0t03x529wz0n80000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wk/fm3ylspx3ml0t03x529wz0n80000gn/T/jieba.cache
Loading model from cache /var/folders/wk/fm3ylspx3ml0t03x529wz0n80000gn/T/jieba.cache
Loading model cost 1.025 seconds.
Prefix dict has been built successfully.
Loading model cost 1.024 seconds.
Prefix dict has been built successfully.
Loading model cost 1.024 seconds.
Prefix dict has been built successfully.


['我', '北海', '你', '章', '智子', '啊', '浸泡', '要', '好', '想', '这', '在', '老', '人', '多', '都', '跑', '剧毒', '不', '吗', '有', '水是', '就', '者', '也', '他', '敬礼', '*', '不是', '对', '面壁', '前进', '大', '人类', '》', '《', '来', '脱水', '罗辑', '地球', '钢印', '说', '航天', ':', '时代', '变', '计划', '四', '三体', '兄弟', '刘', '很', '就是', '思想', '我们', '—', '这个', '模范', '吧', '夫妻', '哈哈哈', '呢', '什么', '一个', '先', '自己', '开始', '还', '这是', '一样', '号', '看', '老子', '谢谢', '主', '!', '那', '影帝', '~', 'eto', '个', '给', '和', 'tm', '未来', '丁仪', '真', '～', '场面', '子', '被', '死', '把', '未', '能', '可以', '陨石', '援', '增', '打', '维德', '?', '已经', '破壁', '好评', '知道', '上', '自然选择', '(', '大海', '去', '哈哈', '怎么', ')', '逃亡', '得', '前方', '大史', '流浪', '先跑', '名', '他们', '高能', '同志', '啥', '路', '技术', '不要', '又', '快', '用', '太空', '致敬', '三', '太', '@', '美国', '牛', '危', '演', '现在', 'doge', '谁', '笑', '爆炸', '逼', '耳机', '=', '让', '一部分', '不在乎', '子弹', '真的', '●', '哈哈哈哈', '中国', '最', '这么', '队友', '更', '懂', '应该', 'mc', '将', '会', '呀', '·', '秦始皇', '那个', '中', '文明', '征途', '这里', '孩子', '恩斯', '翻译', '为什么', '吴岳', '还有', '一'

将词频统计器应用到 `DataFrame` 上：

In [39]:
df_freq = counter_model.transform(df_rmstop)
df_freq.select("filtered", "features").show()

+--------------------------+--------------------+
|                  filtered|            features|
+--------------------------+--------------------+
|                    [脱水]|    (500,[37],[1.0])|
|            [技术, 公有化]|   (500,[126],[1.0])|
|                    [脱水]|    (500,[37],[1.0])|
|                    [浸泡]|     (500,[6],[1.0])|
|                    [空位]|         (500,[],[])|
|  [你, 也, 会, 空中, 劈叉]|(500,[2,24,164],[...|
|                    [脱水]|    (500,[37],[1.0])|
|          [让, 我, 先, 跑]|(500,[0,16,64,148...|
|            [组织生活, 会]|   (500,[164],[1.0])|
|              [当场, 揭穿]|         (500,[],[])|
|                      [演]|   (500,[139],[1.0])|
|                [对号入座]|         (500,[],[])|
|              [狼人, 自刀]|         (500,[],[])|
|[倍, 啥, 速, 我, 都, 一...|(500,[0,15,71,124...|
|                    [浸泡]|     (500,[6],[1.0])|
|              [安排, 姑娘]|         (500,[],[])|
|              [浸泡, 浸泡]|     (500,[6],[2.0])|
|                    [迷惑]|         (500,[],[])|
|           

再用 `IDF` 对词频进行规约化：

In [40]:
idf = IDF(inputCol="features", outputCol="scaled_features")
idf_model = idf.fit(df_freq)
df_scaled = idf_model.transform(df_freq)
df_scaled.select("content", "scaled_features").show(n=15, truncate=50)

+----------------------------------+--------------------------------------------------+
|                           content|                                   scaled_features|
+----------------------------------+--------------------------------------------------+
|                            脱水！|                    (500,[37],[4.366582645384245])|
|                        技术公有化|                    (500,[126],[5.34285608186011])|
|                              脱水|                    (500,[37],[4.366582645384245])|
|                              浸泡|                     (500,[6],[3.591287493215301])|
|                          空位是？|                                       (500,[],[])|
|                    你也会空中劈叉|(500,[2,24,164],[3.0186874171988904,4.196897672...|
|                              脱水|                    (500,[37],[4.366582645384245])|
|                    让  我  先  跑|(500,[0,16,64,148],[2.653155019426097,3.9034518...|
|                        组织生活会|                   (500,[164],

### 模型训练

我们利用得到的特征对弹幕进行 K-means 聚类：

In [41]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(10).setSeed(123)
kmeans.setMaxIter(100)
kmeans.setFeaturesCol("scaled_features")

kmeans_model = kmeans.fit(df_scaled)
kmeans_model.setPredictionCol("cluster_labels")

df_pred = kmeans_model.transform(df_scaled)

23/06/01 16:42:43 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/01 16:42:43 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/06/01 16:42:43 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


此时 `df_pred` 中包含了最终的聚类结果和若干中间变量：

In [42]:
df_pred.printSchema()

root
 |-- cid: long (nullable = true)
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- id: long (nullable = true)
 |-- mode: long (nullable = true)
 |-- post_time: long (nullable = true)
 |-- seg: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- cluster_labels: integer (nullable = false)



查看聚类结果：

In [43]:
df_pred.select("content", "cluster_labels").groupBy("cluster_labels").count().show()

+--------------+-----+
|cluster_labels|count|
+--------------+-----+
|             6|  129|
|             3|   17|
|             5|   11|
|             9|  187|
|             4|   62|
|             7|    4|
|             2|   40|
|             0|11047|
|             8|    1|
|             1|    2|
+--------------+-----+



最后将聚类结果与原始弹幕和剧集相对照：

In [44]:
video_info = spark.read.json("data/lec14-video-data.json", multiLine=True)
video_title = video_info.select("cid", "title")
video_title.show()

+---------+-------------------------------------+
|      cid|                                title|
+---------+-------------------------------------+
|144541892|     【独播】我的三体之章北海传 第1集|
|144541943|     【独播】我的三体之章北海传 第2集|
|160377038|     【独播】我的三体之章北海传 第3集|
|148952771|     【独播】我的三体之章北海传 第4集|
|150894103|     【独播】我的三体之章北海传 第5集|
|153392221|     【独播】我的三体之章北海传 第6集|
|156629080|     【独播】我的三体之章北海传 第7集|
|159982308|     【独播】我的三体之章北海传 第8集|
|162395026|【独播/完结】我的三体之章北海传 第9集|
+---------+-------------------------------------+



In [45]:
df_res = df_pred.select("cid", "content", "cluster_labels").filter("cluster_labels > 0")
df_res.join(video_title, df_res.cid == video_title.cid, "inner").drop(df_res.cid).\
    drop(video_title.cid).sort("cluster_labels").show(n=300)

+----------------------------------------+--------------+--------------------------------+
|                                 content|cluster_labels|                           title|
+----------------------------------------+--------------+--------------------------------+
|                   哇————————————————...|             1|【独播】我的三体之章北海传 第4集|
|                  老—————————航——————...|             1|【独播】我的三体之章北海传 第5集|
|   法师凭借着在国家范围内以及国家人民...|             2|【独播】我的三体之章北海传 第5集|
|                      智子：给我整不会了|             2|【独播】我的三体之章北海传 第3集|
|        中国不会忘记印度尼西亚的所作所为|             2|【独播】我的三体之章北海传 第3集|
|智子在这么近的距离帮我们拍摄，不会融化吗|             2|【独播】我的三体之章北海传 第4集|
|            中国也不会忘记印尼的所作所为|             2|【独播】我的三体之章北海传 第3集|
|                      办公室会不会超标了|             2|【独播】我的三体之章北海传 第3集|
|   中国也不会忘记印度尼西亚对华人的所...|             2|【独播】我的三体之章北海传 第3集|
|          众所周知虫子是永远不会被打败的|             2|【独播】我的三体之章北海传 第1集|
|            中国也不会忘记印尼的所作所为|             2|【独播】我的三体之章北海传 第3集|
|            中国也不会忘记印尼的所作